In [1]:
#import Libraries

import numpy as np
import pandas as pd
import re

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a51976db99ed4e449404373b785ef6f1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='t4qXhrfHX9hiJ-fjcnENjDxnRCm3C5dnsD5HbaiVj_nN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_a51976db99ed4e449404373b785ef6f1.get_object(Bucket='sentimentanalysis-donotdelete-pr-znigxradjugep9', Key='zomato (2).zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile 
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths= unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
pwd

'/home/wsuser/work'

In [5]:
#First import the dataset in the dataset variable
#data_review variable contains the reveiws and ratings 
dataset = pd.read_csv("zomato.csv")
data_review = dataset['reviews_list']

print(dataset['reviews_list'][0])

[('Rated 4.0', 'RATED\n  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.'), ('Rated 4.0', 'RATED\n  I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\nCheers'), ('Rated 2.0', 'RATED\n  Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this place is mostly visi

In [6]:
x = []
y = []

In [7]:
# here we tokenize the rating string and the review string
#loop over all the rows
for row_num in range(0,51717):
    # split the revie text at '()
    lst = data_review[row_num].split("('")
    for i in lst:
        if len(i) > 5:
            if i.find("',") != -1:
                single_rev = i.split("',")
                if len(single_rev[0]) > 2:
                    x.append(single_rev[0])
                if len(single_rev[1]) > 2:    
                    y.append(single_rev[1])



In [8]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 19.2 MB/s eta 0:00:01


In [9]:
#Import the Librariesimport re 
import nltk
nltk.download("stopwords") 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# to store the final rating
rating_final = []
# to store cleaned revies
review_final = []


In [11]:
# the rating string contains words and numbers 
# so we tokenize the numbers only from it and change into float
# for rartings below 2.5 we store the rating as poor
# for ratings between 2.5 and 3.5 the rating as average
# for ratings more than 3.5 tha rating stored as good

for loop in range(0,40000):
    data_x = x[loop]
    data_x = re.sub('[a-zA-Z]', " ", data_x)
    data_x = data_x.split()
    data_x = ''.join(data_x)
    data_x = float(data_x)
    if data_x < 2.5:
        rating_final.append("poor") #poor
    elif data_x >= 2.5 and data_x <= 3.5 :    
        rating_final.append("average") # average
    elif data_x > 3.5:
        rating_final.append("good") #good
        

In [12]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=ee006b349f9a9b25e62c55f78795c7ee2e1fdefc0cc315ce1aeda2b4a70682e0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [13]:
# label encode the Ratings and OneHotEncode for the classification        

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
rating_final = le.fit_transform(rating_final)

rating_final = np.array(rating_final)
rating_final = np.expand_dims(rating_final, axis=1)
        
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
rates = one.fit_transform(rating_final).toarray()

In [14]:
# Here the unnecessary stop words from the reviews lists
# Stemming operations are also done here 

for loop in range(0,40000) : 
    data_y = y[loop]
    data_y = re.sub('[^a-zA-Z]', " ", data_y)
    data_y = data_y.lower()
    data_y = data_y.split()
    data_y = [ps.stem(word) for word in data_y if not word in set(stopwords.words('english'))]
    data_y = ' '.join(data_y)
    review_final.append(data_y)

In [15]:
# count vectorize the reviews according to the unique words    
                    
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 20000)
x_final = cv.fit_transform(review_final).toarray()

In [16]:
# saving the vectorizer which would be used as dictionary.
import pickle
pickle.dump(cv, open('cv.pkl','wb'))

In [17]:
# Split the data into test and train sets

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final,rates, test_size = 0.2, random_state = 0)  

In [18]:
# adding the neuron layers 
# the units in the input layers is equal to the number of unique words
# taken three deeper layers of 2000 units each
# Relu as activation in the hidden layers
# the output layer has 3 units as the one hot encoding has 3 columns
# the classification is in categorical


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense




In [19]:
model = Sequential()

In [20]:
model.add(Dense(units = 13264, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 3, kernel_initializer = 'random_uniform', activation = 'softmax'))

In [21]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [22]:
model.fit(x_train,y_train,batch_size = 128,epochs = 0)

In [23]:
# testing the prediction

y_pred = model.predict(x_test)

text =  "The food is okay. average place "
text = re.sub('[^a-zA-Z]', ' ',text)
text = text.lower()
text = text.split()
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text = ' '.join(text)

y_p = model.predict(cv.transform([text]))

In [39]:
model.save("zomato_2_analysis.h5")

In [40]:
!tar -zcvf SentimentAnalysis-model_new.tgz zomato_2_analysis.h5

zomato_2_analysis.h5


In [41]:
ls

cv.pkl  SentimentAnalysis-model_new.tgz  zomato_2_analysis.h5  zomato.csv


In [42]:
!pip install watson-machine-learning-client --upgrade

In [43]:
from ibm_watson_machine_learning import APIClient
wml_credentials = { "url": "https://us-south.ml.cloud.ibm.com", 
                  "apikey": "b7BpJij5gFeuL2cR1_s14y1Lb7TssAl94vggb0m3LLqh" }
client = APIClient(wml_credentials)

In [44]:
client = APIClient(wml_credentials)

In [45]:
def guid_from_space_name(client, space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]== space_name)['metadata']['id'])

In [46]:
space_uid= guid_from_space_name(client, 'SentimentAnalysis')
print("Space UID "+ space_uid)

Space UID 7e164fd0-b9db-4b39-92fe-71d0982f3f26


In [47]:
client.set.default_space(space_uid)

'SUCCESS'

In [50]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [53]:
software_spec_uid= client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [54]:
model_details = client.repository.store_model(model='SentimentAnalysis-model_new.tgz', meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
}) 
model_id= client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1452: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [55]:
model_id

'03e9816e-0f49-497a-9aff-f78f13da4189'

In [56]:
client.repository.download(model_id,'CNN')

Successfully saved model content to file: 'CNN'


'/home/wsuser/work/CNN'